# DRIVE MOUNTER

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#  DATA EXTRACTION

In [0]:
import pandas as pd
import numpy as np
import json
import csv
import bz2
import os
import sys

In [0]:
def extractor(fle,writer,up_l,down_l):
  total=[]
  source_file=bz2.BZ2File(fle,'r')
  prev='ip'
  temp_l=[]
  for i,line in enumerate(source_file):
    if i<down_l: 
      continue
    if i==up_l: 
      break
    print(i)
    data=json.loads(line)
    if 'from' not in data.keys() or 'dst_addr' not in data.keys():
      continue
    src_addr=data['from']
    dest_addr=data["dst_addr"]
    temp=data['result']
    for iter in temp:
      if "result" not in iter.keys():
        continue
      data_temp=iter['result']
      for ext in data_temp:
        if "from" in ext.keys():
          if prev==str(ext["from"]): continue
          temp_l.append(ext["from"])
          prev=str(ext["from"])
    if len(temp_l)==0:
      continue
    writer.write(str(src_addr)+","+str(dest_addr)+"\n")
    total.append(temp_l)  
    temp_l=[]
  down_l=up_l
  up_l=up_l+20000
  if i<down_l-1:
    flag=1
  else:
    flag=0

  return total,down_l,up_l,flag  
          

In [0]:
def merge(name_w,total):
  df=pd.read_csv(name_w)
  df['Hops']=total
  df.to_csv(name_w,index=False)

In [0]:
def caller_ext():
  Path="drive/My Drive/iitkgp_summer_project"
  files=os.listdir(Path)
  up_l=20000
  down_l=0
  i=files[0]
  fle=Path+"/"+i
  flag=0
  name=Path+'/'+i[-7:-4]
  if os.path.exists(name):
    print("error")
    exit(0)
  else:
    os.mkdir(name)

  counter_n=1
  while(flag==0):
    
    name_w=name+'/IP_rec'+i[-7:-4]+"_"+str(counter_n)+".csv"
    print(name_w)
    text=open(name_w,"w")

    text.write("source,dest\n")
    text.close()
    writer=open(name_w,'a')

    total,down_l,up_l,flag=extractor(fle,writer,up_l,down_l)
    writer.close()
    merge(name_w, total)  
    counter_n+=1

In [0]:
caller_ext()

# DATA PREPROCESSING

In [0]:
import pandas as pd
import json
import ast
import os

In [0]:
def preprocess(fle,path,fldr):
  Path="drive/My Drive/iitkgp_summer_project"
  df=pd.read_csv(fldr+'/'+fle)
  z=0
  df_l=df.values.tolist()
  data={}
  while z<len(df):
    lst=ast.literal_eval(df_l[z][2])
    k=df_l[z][0]
    lst.insert(0,k)
    lst.append(df_l[z][1])
    j=0
    while j<len(lst)-1:
      if data.get(lst[j])==None:
        data[lst[j]]=[lst[j+1]]
      else:
        ips=data.get(lst[j])
        ips.append(lst[j+1])
        data[lst[j]]=ips
        #print(ips)
      j+=1
    
    if data.get(lst[j])==None:
      data[lst[j]]=[]
    z+=1
  dump=path+"/"+fle[:-4]+'.json'
  print(dump)
  if os.path.exists(dump):
    os.remove(dump)
  with open(dump, 'w') as fp:
    json.dump(data, fp)

In [0]:
def preprocess_caller(fldr):
  files=os.listdir(fldr)
  p=fldr+'_json'
  if os.path.exists(p):
    print(error)
    exit(0)
  else:
    os.mkdir(p)
  for i in files:
    #for j in i:
      preprocess(i,p,fldr)



    

In [0]:
Path="drive/My Drive/iitkgp_summer_project"
files=os.listdir(Path)
up_l=20000
down_l=0
i=files[0]
fle=Path+"/"+i
#flag=0
name=Path+'/'+i[-7:-4]
preprocess_caller(name)

drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_1.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_2.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_3.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_4.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_5.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_6.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_7.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_8.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_9.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_10.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_11.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_12.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_13.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_14.json
drive/My Drive/iitkgp_summer_project/300_json/IP_rec300_15.json
drive/My Drive/iitkgp_summer_project/300_json/IP_

# GRAPH CREATION

In [0]:
import json
import pickle
import os

In [0]:
class Graph:
  def __init__(self,g_dict=None):
    if g_dict is None:
      g_dict=[]
    self.g_dict=g_dict

  def getVertices(self):
    return list(self.g_dict.keys())
  
  def findedge(self):
    edge=[]
    for v in self.g_dict:
      for nxt_v in self.g_dict[v]:
        if (nxt_v,v) not in edge:
          edge.append((nxt_v,v))
          print(len(edge))
    return edge
  
  def edges(self):
      return self.findedge()

  def addVertex(self,v):
    if v not in self.g_dict:
      self.g_dict[v]=[]

  def addedge(self,edge):
    edge=set(edge)
    (v1,v2)=tuple(edge)
    if v1 in self.g_dict:
      self.g_dict[v1].append(v2)
    else:
      self.g_dict[v1]=[v2]


In [0]:
def create_graph(fle,path,fldr):
  Path="drive/My Drive/iitkgp_summer_project"
  f=open(fldr+'/'+fle,"r")
  data=json.loads(f.read())
  graph_elements=data
  g=Graph(graph_elements)
  edge=g.edges()
  ver=g.getVertices()
  
  ext_1=fle[:-5]+'_graph_detail.txt'
  file_d=path+'/'+ext_1
  print(file_d)
  detail=open(file_d,'w')
  detail.write(str(ver)+'\n')
  detail.write(str(edge))
  detail.close()


In [0]:
def graph_caller(fldr):
  files=os.listdir(fldr)
  p=fldr[:-5]+'_graph_detail'
  print(p)
  k=0
  if os.path.exists(p):
    print("error")
    exit(0)
  else:
    os.mkdir(p)
    print("created")
  for i in files:
    #for j in i:
    create_graph(i,p,fldr)


In [0]:
Path="drive/My Drive/iitkgp_summer_project"
files=os.listdir(Path)
up_l=20000
down_l=0
i=files[0]
fle=Path+"/"+i
#flag=0
name=Path+'/'+i[-7:-4]+'_json'
print(name)
graph_caller(name)

Streaming output truncated to the last 5000 lines.
55372
55373
55374
55375
55376
55377
55378
55379
55380
55381
55382
55383
55384
55385
55386
55387
55388
55389
55390
55391
55392
55393
55394
55395
55396
55397
55398
55399
55400
55401
55402
55403
55404
55405
55406
55407
55408
55409
55410
55411
55412
55413
55414
55415
55416
55417
55418
55419
55420
55421
55422
55423
55424
55425
55426
55427
55428
55429
55430
55431
55432
55433
55434
55435
55436
55437
55438
55439
55440
55441
55442
55443
55444
55445
55446
55447
55448
55449
55450
55451
55452
55453
55454
55455
55456
55457
55458
55459
55460
55461
55462
55463
55464
55465
55466
55467
55468
55469
55470
55471
55472
55473
55474
55475
55476
55477
55478
55479
55480
55481
55482
55483
55484
55485
55486
55487
55488
55489
55490
55491
55492
55493
55494
55495
55496
55497
55498
55499
55500
55501
55502
55503
55504
55505
55506
55507
55508
55509
55510
55511
55512
55513
55514
55515
55516
55517
55518
55519
55520
55521
55522
55523
55524
55525
55526
55527
55528
55529
5

# PRINTING GRAPH


In [0]:
import networkx as nx
import matplotlib.pyplot as plt

In [0]:
G=nx.Graph()

In [0]:
G.add_nodes_from(ver)
G.add_edges_from(edge)

In [0]:
plt.figure(1,figsize=(70,70)) 
nx.draw_networkx(G,with_labels=False,node_size=200,node_shape='o',style='solid',linewidths=10.0)
plt.savefig("drive/My Drive/iitkgp_summer_project/fig_1.png")
#plt.show()

# ANALYSIS FROM GRAPH

In [0]:
!pip install cymruwhois


In [0]:
import json
import pandas as pd
from cymruwhois import Client
import socket
import os

In [0]:
Path="drive/My Drive/iitkgp_summer_project"
fle='/IP_rec300.json'

In [0]:
def get_number_of_connections(fle,file_w,fldr):
  #Path="drive/My Drive/iitkgp_summer_project"
  f=open(fle,"r")
  data=json.loads(f.read())
  writer=open(file_w,'a')
  for i in data.keys():
    lst=data.get(i)
    l=len(lst)
    writer.write(i+","+str(l)+"\n")
  writer.close()


In [0]:
def count_executer(fle,p,fldr):
  Path="drive/My Drive/iitkgp_summer_project"
  
  
  file_j=fldr+"/"+fle
  ext=p+"/"+fle[:-5]
  #ext=fldr[:-5]
  file_w=ext+'_IP_details.csv'
  writer=open(file_w,'w')
  writer.write('IP,connection\n')
  writer.close()

  get_number_of_connections(file_j,file_w,fldr)

In [0]:
def exec_caller(fldr):
  files=os.listdir(fldr)
  ext=fldr[:-5]
  p=ext+'_IP_detail'
  print(p)
  if os.path.exists(p):
    print("error")
    exit(0)
  else:
    os.mkdir(p)
    print("Created")
  for i in files:
    #for j in i:
      count_executer(i,p,fldr)



In [0]:
Path="drive/My Drive/iitkgp_summer_project"
files=os.listdir(Path)
up_l=20000
down_l=0
i=files[0]
fle=Path+"/"+i
#flag=0
name=Path+'/'+i[-7:-4]+'_json'
print(name)
exec_caller(name)

drive/My Drive/iitkgp_summer_project/300_json
drive/My Drive/iitkgp_summer_project/300_IP_detail
error


In [0]:
fle='/IP_rec300.csv'
df2=pd.read_csv(Path+fle)
print(df2.head())

           source  ...                                               Hops
0  222.241.197.97  ...  ['192.168.33.253', '192.168.33.1', '222.241.19...
1  222.241.197.97  ...  ['192.168.33.1', '222.241.196.1', '61.187.99.2...
2  222.241.197.97  ...  ['192.168.33.253', '192.168.33.1', '222.241.19...
3  222.241.197.97  ...  ['192.168.33.253', '192.168.33.1', '222.241.19...
4  222.241.197.97  ...  ['192.168.33.253', '192.168.33.1', '222.241.19...

[5 rows x 3 columns]


In [0]:
def create_source(fle):
  df2=pd.read_csv(fle)
  print(df2.head())
  i=0
  df_l=df2.values.tolist()
  source={}
  while i<len(df_l):
    if source.get(df_l[i][0]) is None:
      source[df_l[i][0]]=0
    i+=1
  return source


In [3]:
def Mark_ip(fle_ip,fle):
  k=0
  status=[]
  source=create_source(fle)
  while k<len(df):
    print(k)
    if df.iloc[k]['connection']==0:
      status.append('dest')
    elif source.get(df.iloc[k]['IP']) is not None:
      status.append('sender')
    else:
      status.append('Intermediate')
  
    k+=1

  df['Status']=status
  df.to_csv(fle_ip,index=False)



SyntaxError: ignored

In [0]:
def Mark_caller(fldr_ip,fldr_n):
  file_ip=os.listdir(fldr_ip)
  file_n=os.listdir(fldr_n)

  for i in file_ip,file_n:
    print(i)
    print(j)

    Mark_ip(i,j)


In [0]:
df=pd.read_csv(file_w)
c=Client()
asn=[]
as_name=[]

j=48870
k=0
while k<j:
  asn.append(df.iloc[k]['AS_num'])
  as_name.append(df.iloc[k]['AS_name'])
  k+=1
while j<len(df):
  print(j)
  try:
    ip=df.iloc[j]['IP']
  
    print("A")
    r=c.lookup(ip)
    print(r.asn)
    print(r.owner)
    asn.append(r.asn)
    as_name.append(r.owner)
  except:
    print("error")
    asn.append("NA")
    as_name.append("NA")

  j+=1

df['AS_num']=asn
df['AS_name']=as_name
df.to_csv(file_w,index=False)


Streaming output truncated to the last 5000 lines.
A
197119
ASKRENA, KG
54291
A
8449
AS8449-ELCAT, KG
54292
A
3216
SOVAM-AS, RU
54293
A
NA
NA
54294
A
6762
SEABONE-NET TELECOM ITALIA SPARKLE S.p.A., IT
54295
A
NA
NA
54296
A
6762
SEABONE-NET TELECOM ITALIA SPARKLE S.p.A., IT
54297
A
3356
LEVEL3, US
54298
A
3356
LEVEL3, US
54299
A
3356
LEVEL3, US
54300
A
8881
VERSATEL, DE
54301
A
8881
VERSATEL, DE
54302
A
NA
NA
54303
A
2516
KDDI KDDI CORPORATION, JP
54304
A
1299
TELIANET Telia Carrier, SE
54305
A
3491
BTN-ASN, US
54306
A
20712
AS20712 Andrews & Arnold Ltd, GB
54307
A
20712
AS20712 Andrews & Arnold Ltd, GB
54308
A
6830
LGI-UPC formerly known as UPC Broadband Holding B.V., AT
54309
A
NA
NA
54310
A
6830
LGI-UPC formerly known as UPC Broadband Holding B.V., AT
54311
A
6830
LGI-UPC formerly known as UPC Broadband Holding B.V., AT
54312
A
6830
LGI-UPC formerly known as UPC Broadband Holding B.V., AT
54313
A
6830
LGI-UPC formerly known as UPC Broadband Holding B.V., AT
54314
A
1239
SPRINTLINK, U

In [0]:
print(len(asn))

33949


In [0]:
df=pd.read_csv(file_w)
c=Client()
hn=[]

j=0
while j<len(df):
  print(j)
  ip=df.iloc[j]['IP']
  try:
    h=socket.gethostbyaddr(ip)
    hn.append(h[0])
  except:
    hn.append('NA')
  j+=1
df['host']=hn

df.to_csv(file_w,index=False)

Streaming output truncated to the last 5000 lines.
50524
50525
50526
50527
50528
50529
50530
50531
50532
50533
50534
50535
50536
50537
50538
50539
50540
50541
50542
50543
50544
50545
50546
50547
50548
50549
50550
50551
50552
50553
50554
50555
50556
50557
50558
50559
50560
50561
50562
50563
50564
50565
50566
50567
50568
50569
50570
50571
50572
50573
50574
50575
50576
50577
50578
50579
50580
50581
50582
50583
50584
50585
50586
50587
50588
50589
50590
50591
50592
50593
50594
50595
50596
50597
50598
50599
50600
50601
50602
50603
50604
50605
50606
50607
50608
50609
50610
50611
50612
50613
50614
50615
50616
50617
50618
50619
50620
50621
50622
50623
50624
50625
50626
50627
50628
50629
50630
50631
50632
50633
50634
50635
50636
50637
50638
50639
50640
50641
50642
50643
50644
50645
50646
50647
50648
50649
50650
50651
50652
50653
50654
50655
50656
50657
50658
50659
50664
50665
50666
50667
50668
50669
50670
50671
50672
50673
50674
50675
50676
50677
50678
50679
50680
50681
50682
50683
50684
50685
5